In [1]:
import pandas as pd
import regex as re
import numpy as np
import inquirer

In [2]:
# On recupere le fichier nettoye et les categories

data_df = pd.read_csv('Ressources/03_Donnees Nettoyees.csv')

jus_de_fruits_df = pd.read_csv('Ressources/Categories du Panier/jus_de_fruits_df.csv')
tartines_df = pd.read_csv('Ressources/Categories du Panier/tartines_df.csv')
cereales_df = pd.read_csv('Ressources/Categories du Panier/cereales_df.csv')
laits_df = pd.read_csv('Ressources/Categories du Panier/laits_df.csv')
confitures_et_pates_a_tartiner_df = pd.read_csv('Ressources/Categories du Panier/confitures_et_pates_a_tartiner_df.csv')
cafes_df = pd.read_csv('Ressources/Categories du Panier/cafes_df.csv')
thes_df = pd.read_csv('Ressources/Categories du Panier/thes_df.csv')
biscuits_sucres_df = pd.read_csv('Ressources/Categories du Panier/biscuits_sucres_df.csv')
desserts_df = pd.read_csv('Ressources/Categories du Panier/desserts_df.csv')
soupes_df = pd.read_csv('Ressources/Categories du Panier/soupes_df.csv')
pates_df = pd.read_csv('Ressources/Categories du Panier/pates_df.csv')
riz_df = pd.read_csv('Ressources/Categories du Panier/riz_df.csv')
fruits_df = pd.read_csv('Ressources/Categories du Panier/fruits_df.csv')
legumes_df = pd.read_csv('Ressources/Categories du Panier/legumes_df.csv')
sources_de_proteines_df = pd.read_csv('Ressources/Categories du Panier/sources_de_proteines_df.csv')
sauces_df = pd.read_csv('Ressources/Categories du Panier/sauces_df.csv')
biscuits_aperitifs_df = pd.read_csv('Ressources/Categories du Panier/biscuits_aperitifs_df.csv')
matieres_grasses_df = pd.read_csv('Ressources/Categories du Panier/matieres_grasses_df.csv')

matieres_grasses_df.head()

,Code Barre,Nom du Produit,Pays de Manufacture,Region de Manufacture,Latitude GPS,Longitude GPS,Nutriscore,Ecoscore,Marque,Marque de Distribution
0,25110001525,Huile d'olive vierge extra,Italie,NaN,41.871940,12.567380,6.0,49.531594,Monini,Non
1,319110413922,Huile d'olive vierge extra,France,NaN,NaN,NaN,NaN,65.766726,Fan & Jicé,Non
2,637913429156,Huile de colza,France,Hauts-de-France,50.614407,3.118079,NaN,47.450168,Perle du Gy,Non
3,764549058679,Huile d'olive vierge extra,Espagne,NaN,40.463667,-3.749220,6.0,64.472459,villa blanca,Non
4,11972464,Huile d'olive vierge extra,Italie,NaN,41.871940,12.567380,NaN,41.481594,florelli,Non


In [3]:
# On cree une liste avec tous nos jeux de donnees et une liste avec le nom des categories

categ_df_list = [jus_de_fruits_df, tartines_df, cereales_df, laits_df, confitures_et_pates_a_tartiner_df, cafes_df, thes_df, biscuits_sucres_df,
                 desserts_df, soupes_df, pates_df, riz_df, fruits_df, legumes_df, sources_de_proteines_df, sauces_df, biscuits_aperitifs_df, matieres_grasses_df]

catList = ['Jus de Fruits', 'Tartines', 'Cereales', 'Laits', 'Confitures et Pates a Tartiner', 
           'Cafes', 'Thes', 'Biscuits Sucres', 'Desserts', 'Soupes', 'Pates', 'Riz',
           'Fruits', 'Legumes', 'Sources de Proteines', 'Sauces', 'Biscuits Aperitifs', 'Matieres Grasses']

for i in range(len(categ_df_list)):
    categ_df_list[i].name = (catList[i].lower().replace(' ','_') + '_df')

In [4]:
# Petite correction d'erreurs etranges

errorIndex = []

for cat in categ_df_list:
    for i in range(len(cat)):
        if cat.loc[i,'Nom du Produit'] in ['Nom du Produit']:
            errorIndex.append(i)

    cat.drop(errorIndex, inplace = True)
    cat.reset_index(drop=True, inplace=True)
    
for cat in categ_df_list:
    for i in range(len(cat)):
        cat.loc[i,'Latitude GPS'] = float(cat.loc[i,'Latitude GPS'])
        cat.loc[i,'Longitude GPS'] = float(cat.loc[i,'Longitude GPS'])
        cat.loc[i,'Nutriscore'] = float(cat.loc[i,'Nutriscore'])
        cat.loc[i,'Ecoscore'] = float(cat.loc[i,'Ecoscore'])

In [5]:
# Creation de dictionnaires et liste pour avoir les coordonnees GPS des Regions

chefLieuxDict = dict(zip(pd.read_csv('Ressources/Geolocalisation/chefLieux.csv', usecols = ['Région'])['Région'].tolist(),pd.read_csv('Ressources/Geolocalisation/chefLieux.csv', usecols = ['Chef-lieu de région'])['Chef-lieu de région'].tolist()))

regionGPS_df = pd.read_csv('Ressources/Geolocalisation/cities.csv', usecols = ['name','gps_lat','gps_lng'])
regionGPS_df = regionGPS_df[regionGPS_df['name'].isin(list(chefLieuxDict.values()))]
regionGPS_df.sort_values("name", inplace = True) 
regionGPS_df.drop_duplicates(subset ="name", keep = 'first', inplace = True) 
regionGPS_df.reset_index(drop=True, inplace=True)

latitudeDict = {}
longitudeDict = {}
for region in list(chefLieuxDict.keys()):
    latitudeDict [region]= regionGPS_df.loc[regionGPS_df.index[(regionGPS_df['name'] == chefLieuxDict[region])].tolist()[0],'gps_lat']
    longitudeDict [region]= regionGPS_df.loc[regionGPS_df.index[(regionGPS_df['name'] == chefLieuxDict[region])].tolist()[0],'gps_lng']

regionList = ['Auvergne-Rhône-Alpes', 'Bourgogne-Franche-Comté', 'Bretagne', 'Centre-Val de Loire', 'Corse', 'Grand Est', 'Hauts-de-France', 'Île-de-France', 'Normandie',
              'Nouvelle-Aquitaine', 'Occitanie', 'Pays de la Loire', "Provence-Alpes-Côte d'Azur"]

In [6]:
# Fonction pour trouver la region la plus lointaine (DOM TOM exclus)

def find_further_region(region):
    distanceList = []

    for item in regionList:
        distLatCarre = np.square(latitudeDict[item] - latitudeDict[region])
        distLongCarre = np.square(longitudeDict[item] - longitudeDict[region])
        distanceList.append(np.sqrt(distLatCarre + distLongCarre))
    
    proximiteDict = dict(zip(regionList,distanceList))
    
    global furtherRegion
    furtherRegion = (list(proximiteDict.keys())[list(proximiteDict.values()).index(max(proximiteDict.values()))], max(proximiteDict.values()))
    return furtherRegion

In [7]:
# Fonction pour construire le localscore en fonction de la region ou on se trouve

def find_localscore(region, categorie):
    categorie['Localscore'] = ''
    for i in range(len(categorie)):
# Si on a l'information de la region ou l'information du pays (autre que la France)
        if type(categorie.loc[i,'Region de Manufacture']) == str or categorie.loc[i,'Pays de Manufacture'] not in ['France']:
            distLatCarre = np.square(categorie.loc[i,'Latitude GPS'] - latitudeDict[region])
            distLongCarre = np.square(categorie.loc[i,'Longitude GPS'] - longitudeDict[region])
            categorie.loc[i,'Localscore'] = (np.sqrt(distLatCarre + distLongCarre))
        
# Si le produit est manufacture en France
        else:
            categorie.loc[i,'Localscore'] = find_further_region(region)[1]


In [8]:
# Liste des profils types

profilList = ['Petit Prix', 'Bon pour Moi', 'Bon pour la Planete', 'Bon pour la Region', 'Indecis']

prixRatio = {'Nutriscore' : 0.9, 'Ecoscore' : 0.05, 'Localscore' : 0.05}
moiRatio = {'Nutriscore' : 0.8, 'Ecoscore' : 0.15, 'Localscore' : 0.05}
planeteRatio = {'Nutriscore' : 0.1, 'Ecoscore' : 0.7, 'Localscore' : 0.2}
regionRatio = {'Nutriscore' : 0.2, 'Ecoscore' : 0.3, 'Localscore' : 0.5}
indecisRatio = {'Nutriscore' : 0.33, 'Ecoscore' : 0.33, 'Localscore' : 0.34}

profilDict = dict(zip(profilList, [prixRatio,moiRatio,planeteRatio,regionRatio,indecisRatio]))

In [9]:
# Fonction pour classer les produits selon le profil

def find_rank(profil, categorie):
    profilRatio = [profilDict[profil]['Nutriscore'], profilDict[profil]['Ecoscore'], profilDict[profil]['Localscore']]
    
# Pour ne pas avoir de nutriscore negatif ce qui nous embeterait pour le calcul du ratio
    categorie['Nutriscore']= categorie['Nutriscore'].add(abs(categorie['Nutriscore'].min()))
    
# Creation du nouveau score global
    for i in range(len(categorie)):
        if categorie.loc[i,'Nutriscore'] == 0:
            scoreNut = 1
        else:
            scoreNut = categorie['Nutriscore'].min() / categorie.loc[i,'Nutriscore']
            
        scoreEco = categorie.loc[i,'Ecoscore'] / categorie['Ecoscore'].max()
        
        if categorie.loc[i,'Localscore'] == 0:
            scoreLoc = 1
        else:
            scoreLoc = categorie['Localscore'].min() / categorie.loc[i,'Localscore']
        
        categorie.loc[i,'Score pour le Profil'] =  np.average([scoreNut, scoreEco, scoreLoc], weights=profilRatio)

        
# Reorganisation du jeu de donnees
    if profil in ['Petit Prix']:
        categorie.sort_values(['Marque de Distribution', 'Score pour le Profil'], ascending = False, ignore_index = True, inplace = True)
    else:
        categorie.sort_values(['Score pour le Profil'], ascending = False, ignore_index = True, inplace = True)

In [10]:
# Fonction pour faire un panier

def create_basket(region, profil, magasin):
    
    for cat in categ_df_list:
        find_localscore(region, cat)
        find_rank(profil, cat) 
    
    global basket_df
    basket_df = pd.DataFrame(columns = list(categ_df_list[0].columns))
    
    for i in range(len(categ_df_list)):
        if magasin in ['Non']:
            basket_df.loc[i] = dict(categ_df_list[i].loc[0])
        else:
            market_df = categ_df_list[i].loc[ categ_df_list[i]['Marque'] == magasin ]
            market_df.reset_index(drop=True, inplace=True)
            if len(market_df) == 0:
                basket_df.loc[i] = dict(categ_df_list[i].loc[0])
            else:
                basket_df.loc[i] = dict(market_df.loc[0])

In [16]:
# Prototype d'appli

magasinList = ['Boni', 'U', 'Intermarché', 'Auchan', 'Lidl','Franprix', 'Casino', 'Monoprix', 'Carrefour']

region = input("Dans quelle region vous trouvez-vous? \n  \n (%s) \n" %(regionList))
profil = input("\n Quel type de panier desirez-vous? \n \n (%s) \n" %(profilList))
if profil in ['Petit Prix']:
    print('La liste des marques de distribution connues est \n %s \n \n \n ' %(magasinList))
    magasinList.append('Non')
    magasin = input("Avez-vous une marque de choix, si oui laquelle? \n")

create_basket(region, profil, magasin)
panierNomList = list(basket_df['Nom du Produit'])
panierCodeBarreList = list(basket_df['Code Barre'])
panierMarqueList = []
panierImageList = []
for i in range(len(basket_df)):
    panierMarqueList.append(data_df.loc[data_df[data_df['Code Barre'] == str(basket_df.loc[i,'Code Barre'])].index.tolist()[0],'Marque'])
    panierImageList.append(i)

print('Vous etes dans la region %s et vous desirez un panier %s. \n' % (region, profil))
print('Il vous faut donc acheter :')
for item in range(len(panierNomList)):
    if item == len(panierNomList)-1:
        if type(panierMarqueList[item]) == str and type(panierImageList[item]) == str:
            print('Le produit %s, \n      dont le code barre est %s \n      de la marque %s \n      ressemblant a : %s.' %(panierNomList[item],panierCodeBarreList[item],panierMarqueList[item]),panierImageList[item])
        elif type(panierMarqueList[item]) == str and type(panierImageList[item]) != str:
            print('Le produit %s, \n      dont le code barre est %s \n      de la marque %s.' %(panierNomList[item],panierCodeBarreList[item],panierMarqueList[item]))
        elif type(panierMarqueList[item]) != str and type(panierImageList[item]) == str:
            print('Le produit %s, \n      dont le code barre est %s \n      ressemblant a : %s.' %(panierNomList[item],panierCodeBarreList[item],panierImageList[item]))
        else:
            print('Le produit %s, \n      dont le code barre est %s.' %(panierNomList[item],panierCodeBarreList[item]))
    else:
        if type(panierMarqueList[item]) == str and type(panierImageList[item]) == str:
            print('Le produit %s, \n      dont le code barre est %s \n      de la marque %s \n      ressemblant a : %s,' %(panierNomList[item],panierCodeBarreList[item],panierMarqueList[item]),panierImageList[item])
        elif type(panierMarqueList[item]) == str and type(panierImageList[item]) != str:
            print('Le produit %s, \n      dont le code barre est %s \n      de la marque %s,' %(panierNomList[item],panierCodeBarreList[item],panierMarqueList[item]))
        elif type(panierMarqueList[item]) != str and type(panierImageList[item]) == str:
            print('Le produit %s, \n      dont le code barre est %s \n      ressemblant a : %s,' %(panierNomList[item],panierCodeBarreList[item],panierImageList[item]))
        else:
            print('Le produit %s, \n      dont le code barre est %s,' %(panierNomList[item],panierCodeBarreList[item]))

Dans quelle region vous trouvez-vous? 
  
 (['Auvergne-Rhône-Alpes', 'Bourgogne-Franche-Comté', 'Bretagne', 'Centre-Val de Loire', 'Corse', 'Grand Est', 'Hauts-de-France', 'Île-de-France', 'Normandie', 'Nouvelle-Aquitaine', 'Occitanie', 'Pays de la Loire', "Provence-Alpes-Côte d'Azur"]) 
 Bourgogne-Franche-Comté

 Quel type de panier desirez-vous? 
 
 (['Petit Prix', 'Bon pour Moi', 'Bon pour la Planete', 'Bon pour la Region', 'Indecis']) 
 Bon pour Moi


Vous etes dans la region Bourgogne-Franche-Comté et vous desirez un panier Bon pour Moi. 

Il vous faut donc acheter :
Le produit Jus multifruit, pur jus, standard, 
      dont le code barre est 3222471081686 
      de la marque Casino,
Le produit Pain de mie, complet, 
      dont le code barre est 3222471854112 
      de la marque Casino,
Le produit Muesli croustillant aux fruits ou fruits secs, enrichi en vitamines et mineraux, 
      dont le code barre est 3222472589464 
      de la marque Casino,
Le produit Lait entier, pasteurise, 
      dont le code barre est 3222475125072 
      de la marque Casino,
Le produit Confiture de fraise (extra ou classique), 
      dont le code barre est 3222473037322 
      de la marque Casino,
Le produit Cafe au lait ou cappuccino, poudre soluble, 
      dont le code barre est 3222473628490 
      de la marque Casino,
Le produit The infuse, non sucre, 
      dont le code barre est 3222472217732 
      de la marque Casino,
Le produit Biscuit sec petit